# 생활안전지도 api 데이터 수집 코드

In [1]:
import urllib
import requests
import pandas as pd
from lxml import html
from lxml import etree
from bs4 import BeautifulSoup

In [2]:
api_key = 'U4XZYP4K-U4XZ-U4XZ-U4XZ-U4XZYP4KUZ'
service = "WFS"
version = "1.1.1"
request = "GetFeature"
typename = "A2SM_CRMNLSTATS"
# styles = "A2SM_CrmnlHspot_Tot_Brglr"
outputformat = "GML2"
base_url = "http://www.safemap.go.kr/sm/apis.do"
option = "?apikey=%s&SERVICE=%s&VERSION=%s&REQUEST=%s&SRS=EPSG&TYPENAME=%s&OUTPUTFORMAT=%s"
url = base_url + option % (api_key, service, version, request, typename, outputformat)
url

'http://www.safemap.go.kr/sm/apis.do?apikey=U4XZYP4K-U4XZ-U4XZ-U4XZ-U4XZYP4KUZ&SERVICE=WFS&VERSION=1.1.1&REQUEST=GetFeature&SRS=EPSG&TYPENAME=A2SM_CRMNLSTATS&OUTPUTFORMAT=GML2'

In [3]:
resp = requests.get(url)
resp.encoding

'ISO-8859-1'

In [4]:
resp.encoding = 'utf-8'

In [5]:
# Parsing
text = resp.text
soup = BeautifulSoup(text , 'lxml')

/Users/dangchan/miniforge3/envs/spatial/lib/python3.8/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [6]:
# find stats
stats = soup.findAll('safemap:a2sm_crmnlstats')

In [7]:
# to dataframe
header = ['polc_nm','plcstn_nm','polc_se','murder','brglr','rape','theft','violn','arson','nrctc','tmpt','gamble','tot','ctprvn_nm','sgg_kor_nm','ctprvn_cd','sgg_cd','x','y']
body = [header]
for stat in stats:
    line = []
    line.append(stat.find('safemap:polc_nm').text)
    line.append(stat.find('safemap:plcstn_nm').text)
    line.append(stat.find('safemap:polc_se').text)
    line.append(stat.find('safemap:murder').text)
    line.append(stat.find('safemap:brglr').text)
    line.append(stat.find('safemap:rape').text)
    line.append(stat.find('safemap:theft').text)
    line.append(stat.find('safemap:violn').text)
    line.append(stat.find('safemap:arson').text)
    line.append(stat.find('safemap:nrctc').text)
    line.append(stat.find('safemap:tmpt').text)
    line.append(stat.find('safemap:gamble').text)
    line.append(stat.find('safemap:tot').text)
    line.append(stat.find('safemap:ctprvn_nm').text)
    line.append(stat.find('safemap:sgg_kor_nm').text)
    line.append(stat.find('safemap:ctprvn_cd').text)
    line.append(stat.find('safemap:sgg_cd').text)
    line.append(stat.find('safemap:x').text)
    line.append(stat.find('safemap:y').text)
    body.append(line)

df = pd.DataFrame(body)

In [8]:
df.columns = header
df = df.iloc[1:,:]
df # srs = epsg3857

,polc_nm,plcstn_nm,polc_se,murder,brglr,rape,theft,violn,arson,nrctc,tmpt,gamble,tot,ctprvn_nm,sgg_kor_nm,ctprvn_cd,sgg_cd,x,y
1,부산강서,가덕도,03_파출소,1,1,1,1,1,1,1,1,1,1,부산광역시,강서구,26,26440,14341400.35,4171070.16603
2,부산강서,가락,03_파출소,1,1,1,1,1,1,1,1,1,1,부산광역시,강서구,26,26440,14349320.89,4190405.88098
3,부산강서,녹산,03_파출소,1,5,1,1,1,1,1,1,2,1,부산광역시,강서구,26,26440,14347557.88,4179208.11599
4,부산강서,대저,03_파출소,5,5,2,3,2,1,5,1,5,2,부산광역시,강서구,26,26440,14358294.03,4192724.85597
5,부산강서,신호,03_파출소,1,5,3,2,3,1,2,1,5,3,부산광역시,강서구,26,26440,14344230.47,4176101.17504
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2032,서울강남,서울강남,01_경찰서,1,1,1,1,1,1,1,1,1,1,서울특별시,강남구,11,11680,14145041.8,4510346.90396
2033,진해,자은,03_파출소,3,1,2,2,2,3,1,1,2,2,경상남도,창원시 진해구,48,48129,14328285.11,4184832.303
2034,군산,어청도,03_파출소,1,1,2,1,1,1,1,1,1,1,전라북도,군산시,45,45130,14024030.84,4317201.87702
2035,대전유성,복용,03_파출소,2,1,1,1,1,1,1,1,1,1,대전광역시,유성구,30,30200,14172758.2,4347948.67003


In [9]:
# 서울시 해당 데이터만 추출
df = df[df.ctprvn_nm=='서울특별시']
df

,polc_nm,plcstn_nm,polc_se,murder,brglr,rape,theft,violn,arson,nrctc,tmpt,gamble,tot,ctprvn_nm,sgg_kor_nm,ctprvn_cd,sgg_cd,x,y
27,서울중부,을지로3가,03_파출소,1,1,1,1,2,1,2,1,4,2,서울특별시,중구,11,11140,14136777.68,4518316.33103
28,서울중부,을지,02_지구대,5,1,5,4,2,1,4,1,5,4,서울특별시,중구,11,11140,14138014.58,4518363.72903
29,서울중부,충무,03_파출소,1,1,1,1,1,1,1,1,1,1,서울특별시,중구,11,11140,14136636.97,4517569.51304
30,서울중부,약수,02_지구대,5,1,2,2,4,2,5,1,1,3,서울특별시,중구,11,11140,14138954.22,4516353.00002
31,서울중부,광희,02_지구대,5,5,2,5,5,5,5,5,5,5,서울특별시,중구,11,11140,14139066,4518193.91004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1974,서울서부,역촌,03_파출소,1,1,2,3,3,1,1,1,1,3,서울특별시,은평구,11,11380,14128120.61,4523716.59
2022,서울광진,구의,03_파출소,1,5,1,2,2,2,2,1,1,2,서울특별시,광진구,11,11215,14147735.13,4515461.47
2025,서울관악,서울관악,01_경찰서,1,1,1,1,1,1,1,1,1,1,서울특별시,관악구,11,11620,14132128.06,4505456.44496
2029,서울서부,서울서부,01_경찰서,1,1,1,1,1,1,5,1,1,1,서울특별시,은평구,11,11380,14128811.82,4523375.09201


In [25]:
# To geodataframe
import geopandas as gpd

gdf = gpd.GeoDataFrame(df[df.columns.drop(['x','y','ctprvn_cd','sgg_cd','ctprvn_nm'])], geometry=gpd.points_from_xy(x=df.x, y= df.y, crs='epsg:3857'))
gdf = gdf.reset_index(drop=True)

In [27]:
gdf.to_file('safemap.gpkg')